In [1]:
import pandas as pd

In [25]:
db = pd.read_excel(r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\63 сш.xlsx").map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

In [38]:
to_read = {
    1 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы (Done)\Done COPY - Қожагелді батыр мектебі 4,8 сын БЖБ ТЖБ.xlsx",
    2 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы (Done)\Done COPY - сш Маяковского СОР и Соч  1 четв.xlsx",
    3 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - 4-8 И Құтпанұлы Аршалы ауданы Жібек жолы ауылы.xlsx",
    4 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - Ж. Бектұров атындағы №3 ЖОББМ 4-8сынып.xlsx",
    5 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область (1 remaining)\Done COPY - ШОртанды №3 ЖОББМ БЖБ және ТЖБ қорытындысы (1).xlsx",
    6 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - 4,8  сыныптар    СОР и СОЧ  ХРОМТАУ.xlsx",
    7 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY №2 Бадамша ОМ СОР и СОЧ.xlsx",
    8 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Актюбинская область\Done COPY - №4 ШЖОББМ.xlsx"
}

df_dict = {}
ss_list = ['sor1', 'sor2', 'sor3', 'soch']
for key, file_path in to_read.items():
    try:
        df_dict[f"df{key}"] = pd.read_excel(file_path, sheet_name='data', dtype={col: str for col in ss_list})
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file: {file_path}. Error: {e}")

In [39]:
for key, df in df_dict.items():
    df = df.fillna('0')

    df[ss_list] = df[ss_list].map(lambda x: ''.join(x.split()) if isinstance(x, str) else x)

    df['grade'] = df['subject'].str[0]
    df['grade'] = pd.to_numeric(df['grade'], errors='coerce')

    df['subject'] = df['subject'].str[1:]

    df[ss_list] = df[ss_list].map(lambda x: x.replace('\\', ' ').replace('/', ' ').replace('|', ' ').replace('.', '').replace(',', ''))

    for col in ss_list:
        df[f"{col}_max"] = df[col].apply(lambda x: x.split()[1] if (len(x.split()) > 1 and x != '0') else x)
        df[col] = df[col].apply(lambda x: x.split()[0] if len(x.split()) >= 1 else x)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[f"{col}_max"] = pd.to_numeric(df[f"{col}_max"], errors='coerce')
    
    df = df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

    name = df.loc[2, 'fio']
    school = db.loc[db['fio'].str.contains(name), 'school_ru'].unique()
    if len(school) == 1:
        df['school'] = school[0]
    else:
        print(f"Error: '{key}' has no school name")
    
    
    reindexed_cols = ['school', 'fio', 'grade', 'subject', 'sor1', 'sor1_max', 'sor2', 'sor2_max', 'sor3', 'sor3_max', 'soch', 'soch_max']
    df = df.reindex(columns=reindexed_cols)
    df_dict[key] = df
    # display(df)

# Testing
for key, df in df_dict.items():
    if df.isna().values.any():
        print(f"Error: '{key}' contains NaN values")
        display(df[df.isna()].head(3))
    for subject in df['subject']:
        if len(subject) < 4:
            print(f"Error: 'subject' in '{key}' < 4 characters")
            display(df[df['subject'] == subject])
            break
    

Error: 'df4' has no school name
Error: 'df6' has no school name
Error: 'df4' contains NaN values


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Error: 'df6' contains NaN values


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Error: 'subject' in 'df6' < 4 characters


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
445,NaN,Қуанышова Жанерке,0,,0,0,0.0,0.0,0.0,0.0,0,0
446,NaN,Қуанышова Жанерке,0,,0,0,0.0,0.0,0.0,0.0,0,0
460,NaN,Максюк Тахир,0,,0,0,0.0,0.0,0.0,0.0,0,0


Error: 'df8' contains NaN values


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Error: 'subject' in 'df8' < 4 characters


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Шубаркудыкская общеобразовательная средня...",Алмаханова Гаухар Қанатқызы,NaN,ZFL,6,3,7,3,0,0,30,14
8,"КГУ ""Шубаркудыкская общеобразовательная средня...",Аязбек Тұрлыхан,NaN,ZFL,6,3,7,4,0,0,30,20
16,"КГУ ""Шубаркудыкская общеобразовательная средня...",Бахытжан Зәәйнаб Бауыржанқызы,NaN,ZFL,6,3,7,4,0,0,30,15
24,"КГУ ""Шубаркудыкская общеобразовательная средня...",Бисенбай Әли Бекболатұлы,NaN,ZFL,6,3,7,3,0,0,30,17
32,"КГУ ""Шубаркудыкская общеобразовательная средня...",Әбдуали Алихан Аймұханұлы,NaN,ZFL,6,4,7,3,0,0,30,25
40,"КГУ ""Шубаркудыкская общеобразовательная средня...",Жолай Жандос Қуанышұлы,NaN,ZFL,6,3,7,2,0,0,30,15
48,"КГУ ""Шубаркудыкская общеобразовательная средня...",Иван Әлім Талғатұлы,NaN,ZFL,6,5,7,3,0,0,30,16
56,"КГУ ""Шубаркудыкская общеобразовательная средня...",Кабакова Жасмин Жасұланқызы,NaN,ZFL,6,3,7,4,0,0,30,15
64,"КГУ ""Шубаркудыкская общеобразовательная средня...",Қазыбаева Жанайым Нұрланқызы,NaN,ZFL,6,5,7,5,0,0,30,25
72,"КГУ ""Шубаркудыкская общеобразовательная средня...",Қайран Әли Қанатұлы,NaN,ZFL,6,4,7,3,0,0,30,15


In [51]:
# Debugging
for col in ss_list:
    for val in df_dict['df3'][col]:
        print(val)
        break

15 14
15 13
nan
20 18


In [68]:
# # Reading Excel Files
# to_read = {
#     1 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы\COPY - Қожагелді батыр мектебі 4,8 сын БЖБ ТЖБ.xlsx",
#     2 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы\COPY - сш Маяковского СОР и Соч  1 четв.xlsx",
#     3 : r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Акмолинская область\COPY - 4-8 И Құтпанұлы Аршалы ауданы Жібек жолы ауылы.xlsx"
# }

# df_dict = {}
# ss_list = ['sor1', 'sor2', 'sor3', 'soch']
# for key, file_path in to_read.items():
#     try:
#         df_dict[f"df{key}"] = pd.read_excel(file_path, sheet_name='data', dtype={col: str for col in ss_list})
#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#     except Exception as e:
#         print(f"Error reading file: {file_path}. Error: {e}")

# # Data Cleaning
# for key, df in df_dict.items():
#     df[ss_list] = df[ss_list].map(lambda x: ''.join(x.split()) if isinstance(x, str) else x)
#     df['grade'] = df['subject'].str[0]
#     df['subject'] = df['subject'].str[1:]
#     name = df.loc[2, 'fio']
#     school = db.loc[db['fio'].str.contains(name), 'school_ru'].values[0]
#     df['school'] = school
#     for _list in df[ss_list].values:
#         for val in _list:
#             if ('/' in str(val) or '\\' in str(val)) and (set(ss_list).issubset(df.columns)) and(isinstance(val, str)):
#                 df[ss_list] = df[ss_list].map(lambda x: x.replace('\\', ' ').replace('/', ' ').replace('.', '') if isinstance(x, str) else x)
#                 max_dict = {f"{col}_max": df[col].str.split().str[1] for col in ss_list if isinstance(df[col], str)}
#                 df = df.assign(**max_dict)
#                 df[ss_list] = df[ss_list].map(lambda x: x[0] if isinstance(x, str) else x)
                
#                 # Assigning the modified DataFrame back to the dictionary
#                 df_dict[key] = df
#                 break
#         reindexed_cols = ['school', 'fio', 'grade', 'subject', 'sor1', 'sor1_max', 'sor2', 'sor2_max', 'sor3', 'sor3_max', 'soch', 'soch_max']
#         df = df.reindex(columns=reindexed_cols)
#         df_dict[key] = df
#         display(df)
#         break

,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,MATH,5,NaN,6,NaN,6,NaN,8,NaN
1,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,KZFL,6,NaN,5,NaN,NaN,NaN,9,NaN
2,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,4,SCIE,7,NaN,8,NaN,NaN,NaN,1,NaN
3,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,MATH,7,NaN,8,NaN,8,NaN,8,NaN
4,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,4,KZFL,7,NaN,7,NaN,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
750,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,8,ENFL,7,NaN,9,NaN,NaN,NaN,2,NaN
751,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,8,KZSL,6,NaN,8,NaN,NaN,NaN,2,NaN
752,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,8,MATH,8,NaN,NaN,NaN,NaN,NaN,1,NaN
753,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,8,PHYS,7,NaN,1,NaN,NaN,NaN,1,NaN


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""СШ имени Маяковского""",Габидула Рауан,8,CHEM,8,15,7,15,6,15,13,25
1,"КГУ ""СШ имени Маяковского""",Габидула Рауан,8,BIOL,10,15,10,15,8,15,22,25
2,"КГУ ""СШ имени Маяковского""",Габидула Рауан,8,ENFL,6,11,7,14,NaN,0,10,24
3,"КГУ ""СШ имени Маяковского""",Габидула Рауан,8,KZSL,7,13,10,14,NaN,0,13,30
4,"КГУ ""СШ имени Маяковского""",Габидула Рауан,8,MATH,8,14,NaN,0,NaN,0,9,20
...,...,...,...,...,...,...,...,...,...,...,...,...
700,"КГУ ""СШ имени Маяковского""",Хуанышбай Алтынбек,4,SCIE,6,9,6,10,NaN,0,8,18
701,"КГУ ""СШ имени Маяковского""",Хуанышбай Алтынбек,4,KZFL,5,10,5,10,NaN,0,7,18
702,"КГУ ""СШ имени Маяковского""",Хуанышбай Мадина,4,KZFL,8,10,8,10,NaN,0,12,0
703,"КГУ ""СШ имени Маяковского""",Хуанышбай Мадина,4,MATH,8,10,7,10,8,10,12,0


,school,fio,grade,subject,sor1,sor1_max,sor2,sor2_max,sor3,sor3_max,soch,soch_max
0,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Айқынқызы Кәусар,8,CHEM,1,NaN,1,NaN,NaN,NaN,2,NaN
1,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Айқынқызы Кәусар,8,BIOL,1,NaN,1,NaN,NaN,NaN,3,NaN
2,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Айқынқызы Кәусар,8,ENFL,1,NaN,1,NaN,NaN,NaN,2,NaN
3,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Айқынқызы Кәусар,8,KZFL,1,NaN,1,NaN,NaN,NaN,3,NaN
4,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Айқынқызы Кәусар,8,MATH,NaN,NaN,1,NaN,NaN,NaN,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1027,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Үсімбай Ернұр,4,MATH,1,NaN,1,NaN,NaN,NaN,1,NaN
1028,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Үсімбай Ернұр,4,SCIE,8,NaN,1,NaN,NaN,NaN,1,NaN
1029,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Тұрғын Тұмар,4,KZFL,1,NaN,1,NaN,NaN,NaN,1,NaN
1030,"КГУ ""Общеобразовательная школа имени Иманжусуп...",Тұрғын Тұмар,4,MATH,1,NaN,1,NaN,NaN,NaN,1,NaN


In [248]:
# to_read = {
#     1: r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы\COPY - Қожагелді батыр мектебі 4,8 сын БЖБ ТЖБ.xlsx",
#     2: r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы\COPY - сш Маяковского СОР и Соч  1 четв.xlsx"
# }

# for key, file_path in to_read.items():
#     try:
#         df_dict[f"df{key}"] = pd.read_excel(file_path, sheet_name='data', dtype={col: str for col in ss_list})
#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#     except Exception as e:
#         print(f"Error reading file: {file_path}. Error: {e}")

# for key, df in df_dict.items():
#     for _list in df[ss_list].values:
#         for val in _list:
#             if ('/' in str(val) or '\\' in str(val)):
#                 display(key, df.head(10))
#                 break
#         break

'df1'

,school,fio,subject,sor1,sor2,sor3,soch
0,Қожагелді батыр мектебі,Айтмұқаш Марғұлан,4MATH,5\10,6\10,6\10,8\16
1,Қожагелді батыр мектебі,Айтмұқаш Марғұлан,4KZFL,6 \10,5\10,NaN,9\16
2,Қожагелді батыр мектебі,Айтмұқаш Марғұлан,4SCIE,7\10,8\10,NaN,10\16
3,Қожагелді батыр мектебі,Асанова Диана,4MATH,7\10,8\10,8\10,8\16
4,Қожагелді батыр мектебі,Асанова Диана,4KZFL,7\10,7\10,NaN,12\16
5,Қожагелді батыр мектебі,Асанова Диана,4SCIE,8\10,8\10,NaN,14\16
6,Қожагелді батыр мектебі,Болатбек Жанмәди,4MATH,8\10,9\10,8\10,14\16
7,Қожагелді батыр мектебі,Болатбек Жанмәди,4KZFL,8\10,9\10,NaN,12\16
8,Қожагелді батыр мектебі,Болатбек Жанмәди,4SCIE,8\10,10\10,NaN,12\16
9,Қожагелді батыр мектебі,Дюманбек Ислам,4MATH,8\10,9\10,9\10,14\16


In [11]:
to_read = {
    1: r"C:\Users\kassenov_r\OneDrive - AEO NIS\Рабочий стол\Мониторинг\Сельские школы\data\СОЧ 63 мектеп 2023\Абай облысы\COPY - Қожагелді батыр мектебі 4,8 сын БЖБ ТЖБ.xlsx"
}
df_dict = {}
df_dict = {f"df{key}" : pd.read_excel(df, sheet_name='data', dtype={col : str for col in ss_list}) for key, df in to_read.items()} # reading the files

ss_list = ['sor1', 'sor2', 'sor3', 'soch']
# df_dict = {key : df[f"{col}_max"] for col in ss_list for key, df in df_dict.items()} # Checking for presence of _max columns

df_dict = {key : df.assign(**{col: df[col].str.split().str.join('') for col in ss_list}) for key, df in df_dict.items()} # removing any space in ss_list columns
df_dict = {key : df.map(lambda x: " ".join(x.split()) if isinstance(x, str) else x) for key, df in df_dict.items()} # removing whitespaces
df_dict = {key : df.assign(grade = df['subject'].str[0], subject=df['subject'].str[1:]) for key, df in df_dict.items()} # creating grade column and removing first char in 'subject' column
df_dict = {key : df.map(lambda x: x.replace('\\', ' ',).replace('/', ' ',).replace('.', '') if isinstance(x, str) else x) for key, df in df_dict.items()}

max_dict = {f"{col}_max" : df[col].str.split(' ').str[1] for _, df in df_dict.items() for col in ss_list} # creating a dict of max score columns
df_dict = {key: df.assign(**{column: value for column, value in max_dict.items()}) for key, df in df_dict.items()} # creating max score columns
df_dict = {key: df.assign(**{col: df[col].str[0] for col in ss_list}) for key, df in df_dict.items()} # leaving first chars (obtained scores) in score columns

name = [df[df.index == 10]['fio'] for _, df in df_dict.items()][0].values[0] # extracting student name to further extract school name
school = db.loc[db['fio'].str.contains(name), 'school_ru'].unique()[0] # extracting school name
df_dict = {key : df.assign(school = school) for key, df in df_dict.items()} # assigning school name to school column

for key, df in df_dict.items():
    display(df)

,school,fio,subject,sor1,sor2,sor3,soch,grade,sor1_max,sor2_max,sor3_max,soch_max
0,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,MATH,5,6,6,8,4,10,10,10,16
1,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,KZFL,6,5,NaN,9,4,10,10,NaN,16
2,"КГУ ""СШ имени Кожагелды батыра""",Айтмұқаш Марғұлан,SCIE,7,8,NaN,1,4,10,10,NaN,16
3,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,MATH,7,8,8,8,4,10,10,10,16
4,"КГУ ""СШ имени Кожагелды батыра""",Асанова Диана,KZFL,7,7,NaN,1,4,10,10,NaN,16
...,...,...,...,...,...,...,...,...,...,...,...,...
750,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,ENFL,7,9,NaN,2,8,10,10,NaN,24
751,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,KZSL,6,8,NaN,2,8,7,10,NaN,30
752,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,MATH,8,NaN,NaN,1,8,13,NaN,NaN,20
753,"КГУ ""СШ имени Кожагелды батыра""",Тлеубергенова Мөлдір Даниярқызы,PHYS,7,1,NaN,1,8,13,16,NaN,25
